In [ ]:
import pandas as pd
from pathlib import Path
import glob
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 200)

In [ ]:
def load_systems_dataframe(ulx_only=False, beamed=False, half_opening_l_45=False):
    systems_df_path = Path('../data/processed/all_systems_df.csv')
    df = pd.read_csv(systems_df_path)
    if ulx_only:
        df = df[df['Lx'] > 1E39]
    if beamed:
        df = df[df['b'] < 1]
    if half_opening_l_45:
        df = df[df['theta_half_deg'] < 45]
    df = df.reset_index()
    df = df.drop(columns=['index', 'Unnamed: 0'])
    return df

In [ ]:
systems_df = load_systems_dataframe(True, False, False)

In [ ]:
systems_df

In [ ]:
#I only ran these simulations to 0.4 BH_NS ratio before realising my method could be faster.
df_a = pd.read_csv('../data/processed/df_a_full.csv')

In [ ]:
df_a

In [ ]:
BH_NS_RATIO = 0.1
plt.xlabel('alive/dead ratio')
plt.ylabel('Number of systems')
plt.title('Transient ULX distribution')

for BH_NS_RATIO in df_a['BH_NS'].unique():
    df = df_a[(df_a['BH_NS'] == BH_NS_RATIO) & (df_a['ratio']!=0) & (df_a['ratio']!=1)]
    # df = df_a[(df_a['BH_NS'] == BH_NS_RATIO)]
    number_of_systems = len(df)
    plt.hist(df['ratio'], bins=50, label=round(BH_NS_RATIO, 2), density=True)
plt.legend()

In [ ]:
df

In [ ]:
my_list = list(systems_df.is_bh)
Z_list = list(systems_df.Z)

is_bh = [my_list[s] for s in df['system_num']]
Z = [Z_list[s] for s in df['system_num']]
df['is_bh'] = is_bh
df['Z'] = Z

In [ ]:
#I've realised that some of the systems have far less simulations on them than others, this needs to be sorted
print(df['system_num'].value_counts().sort_index())
print('==========')
print(df['system_num'].value_counts().sort_values())

In [ ]:
df['is_bh'].value_counts()

In [ ]:
#Example of poorly sampled system
df[df['system_num'] == 529].hist('inclination')

In [ ]:
inclinations = np.arange(91)
dincls = np.arange(1,46)
number_of_ulxs = 151

repeats = 10

number_of_required_simulations = len(inclinations)*len(dincls)*repeats*number_of_ulxs

time_per_simulation = 0.3
simulation_hours = number_of_required_simulations*time_per_simulation/3600

In [ ]:
simulation_hours

In [ ]:
cols = ['system_num', 'Z', 'is_bh', 'dincl', 'inclination', 'alive', 'dead', 'ratio']
df = df[cols]

In [ ]:
df

In [ ]:
df_sub = df[df['Z'] == 0.0002]
plt.hist2d(df_sub['dincl'], df_sub['ratio'], bins=45)

In [ ]:
systems_df['P_wind_years'] = systems_df['P_wind']/60/60/24/365

# Checking new lightcurve simulation

In [ ]:
csv_files = glob.glob('../src/new_curve_results/*.csv')
res = {}
for file in csv_files:
    res[file] = pd.read_csv(file)
df_new = pd.concat(res.items())